# Introduction

Reinforcement Learning is learning what to do $-$ mapping states to actions $-$ to maximize a reward signal. It is different from *supervised learning* because there are no labels or *right answers* given to the system. Training information is used by the *agent* to evaluate its actions (the correct / optimal actions are not provided), and the agent needs to explore the problem actively to search for better and better actions.

## The k-armed Bandit Problem

Consider the following problem:
* You have k options to choose from / actions to take.
* You get a reward corresponding to chosen action.
* Rewards are chosen from a stationary probability distribution.

The objective is to maximize the total reward over a time period.

## Action Values

In order to solve the bandit problem, we need to know the value of each action. We represent this as a *q-function* or *action-value function*.

$a$ $-$ action
\
$t$ $-$ timestep
\
$A_t$ $-$ action selected at timestep $t$
\
$R_t$ $-$ reward received at timestep $t$
\
$q_*(a)$ $-$ expected reward given that action $a$ is selected

$$ q_*(a) \doteq \mathop{{}\mathbb{E}}[R_t | A_t = a] $$

If we know the value of each action, we can simply select the *greedy action* (action with highest reward) every time. 
\
But we don't know the true action values at any given time. 

We can estimate the value of an action as $Q_t(a)$. The closer $Q_t(a)$ is to $q_*(a)$ the better.

  ### Sample Average method
    
  $$Q_t(a) \doteq \frac{\text{sum of rewards when action a taken prior to timestep t}}{\text{number of times action a taken prior to timestep t}}$$
  
  Then, the greedy action would be $A_t = \underset{a}{argmax}\ Q_t(a)$
  
  As the denominator grows larger, i.e., we obtain more and more information about action $a$, $Q_t(a)$ converges to $q_*(a)$.
  
  ### Incremental updates
  
  For a particular action, let $n$ be the number of times it was selected previously. Then its estimated *action value* before it is selected the next time is given as:
  $$ Q_{n+1} = \frac{1}{n}\sum_{i=1}^{n}R_i $$
  
  It is better to have this in a recursive form so that we don't have to remember all $n$ counts of rewards $R_i$ to calculate the action-value at any given point
  
  $$ Q_{n+1} = \frac{1}{n} \left[ R_n + (n-1).\frac{1}{(n-1)}\sum_{i=1}^{n-1}R_i \right]$$
  
  $$ Q_{n+1} = \frac{1}{n} \left[ R_n + (n-1)Q_n \right] $$

  $$ Q_{n+1} = Q_n + \frac{1}{n} \left( R_n - Q_n \right) $$
  
  This is of the form: __NewEstimate = OldEstimate + StepSize(Target - OldEstimate)__
  
  * Generally, __StepSize__ is denoted by $\alpha$, and in this case, $\alpha = \frac{1}{n}$ (*decaying step size*).
  
  * If we use a constant step size, then recent rewards will have more influence over current estimates than older rewards. This is especially useful when the problem is *non-stationary*, i.e., the true action-values keep changing.
  
  * Step size can take any value in $[0,1]$.

## Exploration vs Exploitation

In Reinforcement Learning, we have to balance exploration of the problem and exploitation of our current knowledge.

__Exploitation__: we choose a greedy action based on our current estimate of the action value
* the right thing to do when we want the highest reward at a time step.
* might keep the agent from finding better rewards since it sticks to current knowledge

__Exploration__: we select one of the non-greedy actions
* the right thing to do when the agent has less knowledge of action-value estimates
* can produce a greater total reward in the long

## Strategies to force Exploration

  ### $\epsilon$-greedy action selection
  
  Since the greedy action always chooses the action with the highest value, other actions will not be sampled well enough (or not at all). A simple alternative is to choose uniformly at random any of the actions with $\epsilon$ probability and the greedy action the rest of the time, i.e., with ($1 - \epsilon$) probability.

  ### Optimistic Initial Values
  
  Another simple way to ensure exploration is to initialize all action-values to a large number (usually greater than the maximum possible action value). This makes sure that all of the actions are sampled before the optimal action is identified.
  
  ### Upper Confidence Bound
  
  This method uses the principle of optimism in the face of uncertainty.
  
  We become more certain of the Q-value for an action the more times we sample for that action. If there is an action that has a low expected reward, but hasn't been sampled many times, and therefore, has a chance of yielding a high reward, we choose it.
  
  Doing so either confirms a high reward or it confirms a low reward. Either way, we become more certain of its true value.
  
  Sutton and Barto give us the following formula for action selection:
  
  $$ A_t \doteq argmax \left[ Q_t(a) + c \sqrt{\frac{ln\ t}{N_t(a)}} \right ] $$
  
  Where the first term exploits current action-values, and the second term creates exploration, and:
  \
  $t$ $-$ number of time steps
  \
  $N_t(a)$ $-$ number of times action a was selected
  \
  $c$ $-$ a constant to adjust the exploration term
  
